# Yeast cells detection evaluation notebook

In [ ]:
import sys
sys.path.append('../')
#sys.path.append('%HOMEPATH%/detectron2') #insert local path to detecron2
import numpy as np

import yeastcells.clustering as clustering
import yeastcells.features as features
import yeastcells.data as data
import yeastcells.model as model
import yeastcells.output as output
import yeastcells.fp as fp
import yeastcells.visualize as visualize

In [ ]:
cd ~

### Load data

In [ ]:
path = '../../slow-data/yeastcells/data/cellstar_benchmark/TestSet1'
filenames = data.load_data(path)

In [ ]:
image = data.read_images_cat(filenames)
np.shape(image)

### Load and run model on data for segmentation

In [ ]:
#set path to model_final.pth
model_path = '/var/tensorflow-logs/p253591/yeast-cell-detection-run-4'

#load model
predictor = model.load_model(model_path, seg_thresh=0.94, device='cuda:0') #set GPU if available otherwise use 'cpu'

In [ ]:
output = [
  {'instances': predictor(frame)['instances'].to('cpu')}
  for frame in image
]

### Run clustering algorithm for tracking

In [ ]:
labels, coordinates = clustering.cluster_cells(output, dmax=5, min_samples=3, eps=0.6, progress=False)

### Load ground truth data and reformat prediction outputs into array

In [ ]:
seg_path = '../../slow-data/yeastcells/data/cellstar_benchmark/TestSet1/GroundTruth_Segmentation.csv'
track_path = '../../slow-data/yeastcells/data/cellstar_benchmark/TestSet1/GroundTruth_Tracking.csv'
gt_s, gt_t = data.get_gt(seg_path, track_path)
pred_s, pred_t = data.get_pred(output, labels, coordinates)

### Match predictions to ground truth

In [ ]:
pred_s_, results_s = evaluation.get_seg_performance(pred_s, gt_s, output, removed_fp=False, index=None)
pred_t_, results_t = evaluation.get_track_performance(pred_t, gt_t, output, removed_fp=False, index=None)

In [ ]:
print(results_s)
print(results_t)

In [ ]:
metrics_s = evaluation.calculate_metrics(results_s, pred_s_, gt_s)
metrics_t = evaluation.calculate_metrics(results_t, pred_t_, gt_t)

In [ ]:
print(metrics_s)
print(metrics_t)

In [ ]:
# ## False positive removal

# ### Include features

# # Add std of size and std of position
# areas, g_rate = features.get_area_and_growth_rate(polygons, labels, pred_s)
# X = np.vstack((areas, g_rate)).T

# ### Import random forest and run on prediction features to output idx of false positives

# idx = fp.remove_by_rf(rf, X)

# ### Rerun evaluation with removed false positives

# pred_s_, results_s = evaluation.get_seg_performance(pred_s, gt_s, output, removed_fp=True, index=idx)
# pred_t_, results_t = evaluation.get_track_performance(pred_t, gt_t, output, removed_fp=True, index=idx)